In [ ]:
import numpy as np
 
new_txt = []
file_name = 'A2.txt'
processed_file_name = 'A2_processed.txt'
with open(file_name, 'r') as ori_file:
    with open(processed_file_name, 'w') as proced_file:
        for i, line in enumerate(ori_file):
            if len(line)>57:
                new_line = line[:56]+'\n'
            else:
                new_line = line            
            proced_file.write(new_line)

In [ ]:
import pandas as pd
 
file_name = 'A2.txt'
processed_file_name = 'A2_processed.txt'
csv_name = 'A2_processed_cleaned.csv'
 
data = []
chunk = []
row = 0
 
with open(file_name, 'r') as ori_file:
    with open(processed_file_name, 'w') as proced_file:
        for i, line in enumerate(ori_file):
            if len(line)>57:
                new_line = line[6:56]+'\n'
            else:
                new_line = line[6:]
            if line == '\n':
                if len(chunk)>300:
                    print(f"rwo:{row}, len(chunk):{len(chunk)}")
                data.append(chunk)
                row +=1
                chunk = []
            else:
                new_line = new_line.strip()
                chunk += new_line.split(' ')
 
df = pd.DataFrame(data)    
df.to_csv(csv_name, index=False, header=False)

In [ ]:
import pandas as pd

# Path to the CSV file
file_path = 'A2_processed_cleaned.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Drop the first 54 columns (just extract the Modbus Packets)
df = df.iloc[:, 54:]

# Optionally, save the modified DataFrame to a new CSV file
df.to_csv('A2_processed_cleaned2.csv', index=False)

In [ ]:
import pandas as pd

# Load the first CSV file
df1 = pd.read_csv('information.csv')

# Load the second CSV file
df2 = pd.read_csv('A2_processed_cleaned2.csv')

# Concatenate the two dataframes side by side
combined_df = pd.concat([df1, df2], axis=1)

# Save the combined dataframe to a new CSV file
combined_df.to_csv('A2_processed_cleaned2_combined.csv', index=False)

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('A2_processed_cleaned2_combined.csv')

# Specify the column name
column_name = 'Info'  # Replace with the actual column name

# Filter the DataFrame to keep only rows containing "Query"
filtered_df = df[df[column_name].str.contains('Query', case=False, na=False)]

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv('A2_processed_cleaned2_combined_filtered_file_final(Query).csv', index=False)

# Optionally, print the filtered DataFrame to verify the results
#print(filtered_df)

In [ ]:
import re
import csv

def extract_hex_data_from_log(log_filename):
    hex_data_list = []
    send_pattern = re.compile(r'SEND: (.*)')  # Regex pattern to match the SEND lines and capture hex data

    with open(log_filename, 'r') as log_file:
        for line in log_file:
            match = send_pattern.search(line)
            if match:
                hex_data = match.group(1)  # Extract the hexadecimal data
                hex_data_list.append(hex_data)
    
    return hex_data_list

In [ ]:
def write_hex_data_to_csv(hex_data_list, csv_filename):
    with open(csv_filename, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Hexadecimal Data"])  # Write the header
        for hex_data in hex_data_list:
            writer.writerow([hex_data])  # Write each hex data as a new row

In [ ]:
import pandas as pd

# Specify the names of your input and output CSV files
input_csv_name = 'hex_data_final.csv'  
output_csv_name = 'hex_data_final_processed.csv'  

# Read the original CSV file
df = pd.read_csv(input_csv_name)


packet_column_index = 0  

# Process each row in the dataframe
processed_data = []
for index, row in df.iterrows():
    packet = row[packet_column_index]  # Get the packet from the specified column
    bytes_list = packet.split()  # Split the packet into individual bytes
    processed_data.append(bytes_list)  # Append the list of bytes as a new row in the processed data

# Create a new dataframe from the processed data
processed_df = pd.DataFrame(processed_data)

# Write the processed dataframe to a new CSV file
processed_df.to_csv(output_csv_name, index=False, header=True)


In [ ]:
# padding
import csv

def pad_csv_rows(input_file, output_file):
    # Read all rows from the input CSV file
    with open(input_file, 'r', newline='') as infile:
        reader = csv.reader(infile)
        rows = list(reader)

    # Find the maximum row length
    max_length = max(len(row) for row in rows)

    # Pad each row to the maximum length
    padded_rows = [row + [''] * (max_length - len(row)) for row in rows]

    # Write the padded rows to the output CSV file
    with open(output_file, 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(padded_rows)

# Example usage
input_file = 'A2_processed_cleaned2_combined_filtered_file_final(Query).csv'
output_file = 'A2_processed_cleaned2_combined_filtered_file_padded_final(Query).csv'
pad_csv_rows(input_file, output_file)


In [ ]:
# convert hexadecimal to decimal
import pandas as pd

def hex_to_decimal(hex_str):
    
    return int(hex_str, 16)


input_csv_name = 'A2_processed_cleaned2_combined_filtered_file_padded_final(Query).csv'  
output_csv_name = 'data_final.csv'  

# Read the input CSV file
df = pd.read_csv(input_csv_name)

# Apply the conversion to each cell in the DataFrame
converted_df = df.applymap(lambda cell: hex_to_decimal(cell) if pd.notnull(cell) else cell)

# Write the converted DataFrame to a new CSV file
converted_df.to_csv(output_csv_name, index=False)

print(f"Conversion complete. Decimal values saved to '{output_csv_name}'.")